# Execution environment

In [1]:
!pip install kaggle-environments --upgrade
print("Import started")
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import *
import random
import numpy as np
from scipy.optimize import linear_sum_assignment
print("Import ended")

     |████████████████████████████████| 91 kB 2.3 MB/s eta 0:00:011
^C
ERROR: Operation cancelled by user
Import started
Import ended


# Test Environment

In [2]:
environment = make("halite", configuration={"size": 10, "startingHalite": 5000}, debug=True)
agent_count = 2
environment.reset(agent_count)
state = environment.state[0]
board = Board(state.observation, environment.configuration)

# Framework

## Static
Static

## Navigation
Contains helper functions related to *Points* and *Movement*

## Calculator
Encodes *Board* to numpy array and runs most computationally intensive calculations and heuristics.

In [3]:
# Static
init = False
nav, calc = None, None


class Navigation:
    def __init__(self, board: Board):
        self.CFG = board.configuration

    def dist(self, a: Point, b: Point) -> int:
        return min(abs(a.x - b.x), self.CFG.size - abs(a.x - b.x)) + min(abs(a.y - b.y), self.CFG.size - abs(a.y - b.y))

    def direction_to(self, s: Point, t: Point) -> ShipAction:
        candidate = []  # [N/S, E/W]
        if s.x - t.x != 0:
            candidate.append(ShipAction.WEST if (s.x - t.x) % self.CFG.size < (t.x - s.x) % self.CFG.size else ShipAction.EAST)
        if s.y - t.y != 0:
            candidate.append(ShipAction.SOUTH if (s.y - t.y) % self.CFG.size < (t.y - s.y) % self.CFG.size else ShipAction.NORTH)
        return random.choice(candidate) if len(candidate) > 0 else None

    def unpack(self, n):
        return Point(n // self.CFG.size, n % self.CFG.size)


class Calculator:
    def __init__(self, board: Board):
        self.CFG = board.configuration
        self.me = board.current_player_id
        self.playerNum = len(board.players)

    def update(self, board: Board):
        # Updates
        self.board = board

        # Encoding
        self.encode()

        # Calculate
        self.haliteMean = np.mean(self.haliteMap, axis=None)
        self.ally = self.shipMap[self.me]
        self.enemy = np.sum(self.shipMap, axis=0) - self.ally

    # Encodes halite and units to matrices
    def encode(self) -> dict:
        # Map
        self.haliteMap = np.zeros((self.CFG.size, self.CFG.size))
        self.shipMap = np.zeros((self.playerNum, self.CFG.size, self.CFG.size))
        self.passableMap = np.zeros((self.playerNum, self.CFG.size, self.CFG.size))
        self.shipyardMap = np.zeros((self.playerNum, self.CFG.size, self.BOARD_SIZE))
        for cell in self.board.cells.values():
            self.haliteMap[cell.position.x][cell.position.y] = cell.halite
        for ship in self.board.ships.values():
            self.shipMap[ship.player_id][ship.position.x][ship.position.y] = 1
            if ship.halite == 0:
                self.passableMap[ship.player_id][ship.position.x][ship.position.y] = 1
        for shipyard in self.board.shipyards.values():
            self.shipyardMap[shipyard.player_id][shipyard.position.x][shipyard.position.y] = 1

        # TODO: Add encoding for individual ships and yards (not necessary now)

    # Calculations
    def haliteMapFiltered(self): # I think it's not necessary. 
        self.filteredHaliteMap = np.where(self.haliteMap > mean / 2, self.haliteMap, 0)

    def controlMap(self): # TODO: rename or refactor
        self.controlMap = self.ally - self.enemy
        # TODO: avg pooling


# Agent

In [ ]:
def cost(ship, cell):
    # TODO: much to improve
    cfg = environment.configuration
    haliteCoef = cfg.size / cfg.maxCellHalite
    return nav.dist(ship.position, cell.position) - haliteCoef * cell.halite

@board_agent
def agent(board):
    global init, nav, calc
    if not init:
        init = True
        nav = Navigation(board)
        calc = Calculator(board)

    # Process map
    calc.update(board)
    ships = board.current_player.ships
    shipyards = board.current_player.shipyards

    # Decide tasks 
    miningCells = calc.haliteMap

        # Terrible mining algorithm, should probably come up with something entirely new
    assign = []
    for i, ship in enumerate(ships):
        if ship.cell.halite >= calc.haliteMean / 2:
            pass
        else:
            if ship.halite > 500 and len(shipyards) > 0:
                ship.next_action = direction_to(ship.position, shipyards[0].position)
            else:
                assign.append(ship)

    miningCells = np.argpartition(miningCells, -len(assign),axis=None)[-len(assign):]
    miningCells = miningCells.tolist()
    miningCells = [board.cells[nav.unpack(i)] for i in miningCells]

    costMatrix = np.array([[cost(ship, cell) for ship in assign] for cell in miningCells])
    tasks, _ = linear_sum_assignment(costMatrix)
    j = 0
    for ship in assign:
        ship.next_action = nav.direction_to(ship.position, miningCells[tasks[j]].position)
        j+=1

    if len(shipyards) == 0:
        ships[0].next_action = ShipAction.CONVERT
    for shipyard in shipyards:
        if shipyard.cell.ship is None:
            shipyard.next_action = ShipyardAction.SPAWN

# Run

In [ ]:
environment.reset(agent_count)
environment.run([agent, "random"])
environment.render(mode="ipython", width=500, height=450)

In [ ]:
environment.configuration